## 第零部分:初始化

### 1.匯入函式庫

In [1]:
import pandas as pd #數據檢視
import numpy as np #數學運算

import glob #搜尋檔案

from joblib import load #載入模型

import os #印出檔名

### 2.指定資料位置
* model_path 請選擇需要的模型檔案

In [2]:
data_path ="./test/" #測試數據位置
model_path ="./model/lr.joblib" #預訓練模型位置

### 3.定義函式

In [3]:
def read(filename):
    data = pd.read_csv(filename,skiprows=np.arange(0,3),usecols = np.arange(0,7))
    data.timestamp = (data.timestamp - data.timestamp.iloc[0])/1000
    return data
def sample(data):
    sample_data = pd.DataFrame(data, columns = data.columns).drop(columns='timestamp').astype('float16')
    return sample_data
def feature_extraction(sample_data):
    ax = sample_data.columns[3]
    ay = sample_data.columns[4]
    az = sample_data.columns[5]
    wx = sample_data.columns[0]
    wy = sample_data.columns[1]
    wz = sample_data.columns[2]
    min_ax = np.min(sample_data[ax])
    max_ax = np.max(sample_data[ax])
    mean_ax = np.mean(sample_data[ax])
    std_ax = np.std(sample_data[ax])
    min_ay = np.min(sample_data[ay])
    max_ay = np.max(sample_data[ay])
    mean_ay = np.mean(sample_data[ay])
    std_ay = np.std(sample_data[ay])
    min_az = np.min(sample_data[az])
    max_az = np.max(sample_data[az])
    mean_az = np.mean(sample_data[az])
    std_az = np.std(sample_data[az])
    min_wx = np.min(sample_data[wx])
    max_wx = np.max(sample_data[wx])
    mean_wx = np.mean(sample_data[wx])
    std_wx = np.std(sample_data[wx])
    min_wy = np.min(sample_data[wy])
    max_wy = np.max(sample_data[wy])
    mean_wy = np.mean(sample_data[wy])
    std_wy = np.std(sample_data[wy])
    min_wz = np.min(sample_data[wz])
    max_wz = np.max(sample_data[wz])
    mean_wz = np.mean(sample_data[wz])
    std_wz = np.std(sample_data[wz])
    
    X = [min_ax, max_ax, mean_ax, std_ax, \
                  min_ay, max_ay, mean_ay, std_ay, \
                  min_az, max_az, mean_az, std_az, \
                  min_wx, max_wx, mean_wx, std_wx, \
                  min_wy, max_wy, mean_wy, std_wy, \
                  min_wz, max_wz, mean_wz, std_wz]
    return X

### 4.載入模型

In [4]:
model = load(model_path)

### 5.載入評估數據

In [5]:
type_list = ["原地走","插秧","大跳"] #動作類型
eval_action = [0,1,2] #動作順序
eval_time = [20, 40, 60] #動作結束時間

## 第一部分:評估精確度

### 1.輸出評估數據

In [6]:
interval = 2 #判斷間格時間
acc_sum = 0
for filename in glob.glob(data_path + "*.csv"):
    error = 0 
    print("檔案名稱:" + os.path.basename(filename))
    print("錯誤項目:")
    print("\t" + "開始時間\t結束時間\t實際動作\t預測動作")
    
    data = read(filename) #讀取數據
    fragment = np.floor(max(data.timestamp))/interval #依時間分段
    for idx in np.arange(0,fragment):
        #切割數據
        data_slice = data[(data.timestamp > idx*interval) & (data.timestamp < (idx+1)*interval)] 
        sample_data = sample(data_slice)
        #資料處理
        X = feature_extraction(sample_data)
        X = np.reshape(X,(1,24))
        X_df = pd.DataFrame(X)
        #預測結果
        y = model.predict(X_df)
        #找出這段時間的實際動作
        try:
            action = eval_action[next(x[0] for x in enumerate(eval_time) if x[1] > (idx+1)*interval)]
        except StopIteration:
            pass
        #比較實際動作與預測動作，有錯誤則印出錯誤點
        if action != y[0]:
            error = error + 1 #累加錯誤數量
            print("\t" + str(round(data.timestamp[data.timestamp > (idx)*interval].iloc[0],2)) + \
                  "\t" + str(round(data.timestamp[data.timestamp > (idx+1)*interval].iloc[0],2)) + \
                  "\t" + type_list[action]  + "\t" + "\x1b[31m" + type_list[y[0]] + "\x1b[0m")
    #計算精確度
    acc = (fragment-error)/fragment*100
    acc_sum = acc_sum + acc
    print(f"精確度:{acc:.2f}%")
    print("-"*40)
acc_mean = acc_sum/len(glob.glob(data_path + "*.csv"))
print(f"平均精確度:{acc_mean:.2f}%")

檔案名稱:HIMU-2020-07-24_17-22-00.csv
錯誤項目:
	開始時間	結束時間	實際動作	預測動作
	18.02	20.04	插秧	原地走
	38.06	40.08	大跳	原地走
	58.01	60.02	大跳	原地走
精確度:90.00%
----------------------------------------
檔案名稱:HIMU-2020-07-24_17-23-15.csv
錯誤項目:
	開始時間	結束時間	實際動作	預測動作
	18.04	20.05	插秧	原地走
	38.09	40.1	大跳	原地走
精確度:93.33%
----------------------------------------
檔案名稱:HIMU-2020-07-24_17-24-33.csv
錯誤項目:
	開始時間	結束時間	實際動作	預測動作
	18.04	20.06	插秧	原地走
	38.1	40.01	大跳	原地走
	50.09	52.0	大跳	插秧
	58.05	60.06	大跳	原地走
精確度:86.67%
----------------------------------------
檔案名稱:HIMU-2020-07-24_17-41-40.csv
錯誤項目:
	開始時間	結束時間	實際動作	預測動作
	18.03	20.05	插秧	原地走
	38.08	40.1	大跳	原地走
	40.1	42.01	大跳	原地走
精確度:90.00%
----------------------------------------
檔案名稱:HIMU-2020-07-24_17-42-50.csv
錯誤項目:
	開始時間	結束時間	實際動作	預測動作
	18.03	20.05	插秧	原地走
	38.08	40.1	大跳	插秧
	40.1	42.01	大跳	原地走
	60.05	62.06	大跳	原地走
精確度:87.10%
----------------------------------------
平均精確度:89.42%
